In [31]:
import joblib
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats
from statsmodels.tsa.api import VAR

from tsa import config
from tsa.modeling import ExpandingWindowForecaster

In [32]:
df = pd.read_csv(
    config.PROCESSED_DATA_DIR / "combined_stationary.csv", index_col=0, parse_dates=True
)
df.head()

,zwex_return,impi_return,inflation_rate,rate_change,vacancy_rate,gdp_growth,population_change_d
1993-07-01,-0.373241,-2.037327,-0.028898,-0.299,0.30,0.381316,-0.631895
1993-10-01,-0.307397,0.175953,0.506175,-0.338,0.30,0.547234,-0.017975
1994-01-01,1.156292,1.570735,0.107094,-0.002,0.42,0.562475,0.934411
1994-04-01,1.400267,-0.382496,0.459923,0.856,0.42,-0.831934,-0.168245
1994-07-01,1.249413,-0.496269,-0.481443,0.211,0.42,0.821735,-0.466957


## Model C: VAR with AIC Selection
We estimate a Vector Autoregression (VAR) model using all variables and select the optimal lag length via AIC.

In [33]:
# Fit VAR and select optimal lag using AIC
model = VAR(df)
lag_selection = model.select_order(maxlags=8)
print(lag_selection.summary())

 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0      -9.533      -9.368   7.243e-05      -9.466
1      -12.77     -11.45*   2.862e-06      -12.23
2      -12.81      -10.35   2.751e-06      -11.81
3      -13.72      -10.10   1.139e-06     -12.25*
4     -13.88*      -9.117  1.004e-06*      -11.95
5      -13.73      -7.817   1.249e-06      -11.33
6      -13.69      -6.622   1.450e-06      -10.82
7      -13.71      -5.493   1.654e-06      -10.37
8      -13.60      -4.235   2.286e-06      -9.800
-------------------------------------------------


/Users/liamtessendorf/Programming/Uni/2_Master/5_HS25_Programming/tsa/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)


In [34]:
optimal_lag = lag_selection.aic
print(f"Optimal lag (AIC): {optimal_lag}")
print(f"Optimal lag (BIC): {lag_selection.bic}")

Optimal lag (AIC): 4
Optimal lag (BIC): 1


we will use a lag of 4 as AIC recommends, since this is more common with quarterly data.

In [35]:
var_result = model.fit(optimal_lag)
var_result.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 28, Jan, 2026
Time:                     16:53:39
--------------------------------------------------------------------
No. of Equations:         7.00000    BIC:                   -9.15440
Nobs:                     122.000    HQIC:                  -11.9250
Log likelihood:          -165.747    FPE:                1.06215e-06
AIC:                     -13.8201    Det(Omega_mle):     2.38707e-07
--------------------------------------------------------------------
Results for equation zwex_return
                            coefficient       std. error           t-stat            prob
-----------------------------------------------------------------------------------------
const                          1.687874         0.584010            2.890           0.004
L1.zwex_return                -0.262173         0.100270           -2.615           0.009
L1.impi_return    

### Granger Causality Tests
Test if local variables Granger-cause ZWEX returns (controlling for macro variables).

In [36]:
for var in ["vacancy_rate", "population_change_d"]:
    result = var_result.test_causality("zwex_return", var)
    print(f"{var} -> zwex_return:")
    print(f"    F-statistic: {result.test_statistic:.4f}")
    print(f"    p-value:     {result.pvalue:.4f}")

vacancy_rate -> zwex_return:
    F-statistic: 2.3455
    p-value:     0.0534
population_change_d -> zwex_return:
    F-statistic: 0.1912
    p-value:     0.9430


we can see that both variables are non-significant for ZWEX returns at the 5% level, however, vacancy_rate is on the 10% level.

### Expanding Window Forecast with VAR

In [37]:
forecaster = ExpandingWindowForecaster(initial_window=0.6, random_state=42)
pred_var, actual_var, idx_var = forecaster.var(df_var, target_col="zwex_return")

/Users/liamtessendorf/Programming/Uni/2_Master/5_HS25_Programming/tsa/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
/Users/liamtessendorf/Programming/Uni/2_Master/5_HS25_Programming/tsa/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
/Users/liamtessendorf/Programming/Uni/2_Master/5_HS25_Programming/tsa/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)
/Users/liamtessendorf/Programming/Uni/2_Master/5_HS25_Programming/tsa/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information w

In [38]:
mse_var = mean_squared_error(actual_var, pred_var)
rmse_var = np.sqrt(mse_var)
r2_var = r2_score(actual_var, pred_var)

print(f"Forecast period: {idx_var[0]} to {idx_var[-1]} ({len(idx_var)} observations)")
print(f"MSE:  {mse_var:.6f}")
print(f"RMSE: {rmse_var:.6f}")

Forecast period: 2012-04-01 00:00:00 to 2024-10-01 00:00:00 (51 observations)
MSE:  3.164738
RMSE: 1.778971


In [39]:
forecast_results = {
    "predictions": pred_var,
    "actuals": actual_var,
    "index": idx_var,
    "mse": mse_var,
    "rmse": rmse_var,
}

joblib.dump(forecast_results, config.MODELS_DIR / "var_forecast_results.joblib")

['/Users/liamtessendorf/Programming/Uni/2_Master/5_HS25_Programming/tsa/models/var_forecast_results.joblib']